## The Flyweight pattern

The Flyweight pattern is a memory optimization pattern.
The Flyweight pattern ensures that objects that share a state can use the same
memory for their shared state.
Each Flyweight object has no specific state of its own.
Any time it needs to perform
an operation on SpecificState, that state needs to be passed into the Flyweight by
the calling code as an argument value.


![](uml/flyweight_pattern.png)



### A Flyweight example

534

In [1]:
if __name__ == '__main__':        
    import doctest
    import subprocess
    name = "12-The Flyweight pattern"
    doctest.testmod(verbose=False)
    subprocess.run(f'jupyter nbconvert --to script --output test "{name}"', shell=True)
    std_out = subprocess.run('mypy --strict test.py', capture_output=True, shell=True).stdout
    print(std_out.decode('ascii'))

Success: no issues found in 1 source file

